In [ ]:
import os
import json
from dotenv import load_dotenv, find_dotenv
from datetime import datetime, timedelta
from crewai import Agent, Task, Crew, Process
from crewai_tools import CSVSearchTool
from langchain.tools import tool


: 

In [ ]:
from langchain_openai import ChatOpenAI

load_dotenv(find_dotenv())

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.4
)

In [ ]:
csv_imoveis = CSVSearchTool(csv="files/imoveis.csv")

In [ ]:
#Agente Corretor de Imóveis
corretor = Agent(
    role="Corretor de Imóveis",
    goal="obtenha as melhores recomendações de imóveis para o cliente de acordo com suas preferências com relação ao banco de dados de imóveis disponíveis.",
    backstory="Você é um corretor de imóveis especializado que ajuda os clientes a encontrar o imóvel ideal com base em suas preferências e necessidades.",
    verbose=True,
    max_iter=5,
    tools=[csv_imoveis],
    allow_delegation=False,
    memory=True,
    llm=llm
)

In [ ]:
#Tarefa de Buscar Imóveis
buscar_imoveis = Task(
    description="Busque imóveis que atendam às preferências do cliente, considerando faixas de preço e tipo de imóvel.",
    expected_output="lista de imóveis disponíveis que atendem às preferências do cliente",
    agent=corretor,
    llm=llm
)

In [ ]:
from crewai.tools import BaseTool

def obter_precos_imoveis(cidade: str = "geral"):
    precos = {
        "São Paulo": {"tendencia":"aumento", "percentual":5.2 },
        "Rio de Janeiro": {"tendencia":"estavel", "percentual":0.0 },
        "Belo Horizonte": {"tendencia":"queda", "percentual":-3.1 },
        "geral": {"tendencia":"aumento", "percentual":4.0 }
    }
    return precos.get(cidade, precos["geral"])

class TendenciasPrecoImoveisTool(BaseTool):
    name: str = "analista de preço imobiliários"
    description: str = "Obtenha as tendências de preços de imóveis em diferentes cidades."

    def _run(self, cidade: str) -> dict:
        """
        Executa análises de preços imobiliários e retorna a tendência com base na cidade.
        """

        try:
            return obter_precos_imoveis(cidade)
        except Exception as e:
            return {"error": f"erro ao obter tendencias de preços: {str(e)}"}

In [ ]:
analista_mercado= Agent(
    role="Analista de Mercado Imobiliário",
    goal="Fornecer informações sobre tendências de preços de imóveis na cidade solicitada.",
    backstory="Você é um analista de mercado imobiliário que fornece insights sobre tendências de preços de imóveis em diferentes cidades.",
    verbose=True,
    max_iter=5,
    allow_delegation=False,
    memory=True,
    llm=llm
)

In [ ]:
obter_tendencias= Task(
    description="""
    Analise as tendências de preços de imóveis na cidade {cidade} e forneça insights sobre a situação do mercado imobiliário 
    (Valorizado ou Desvalorizado). Considere o tipo de imóvel {imovel} e a faixa de preço {faixa_preco}.
    """,
    expected_output="Resumo das tendências de preços de imóveis na cidade solicitada, incluindo a situação do mercado imobiliário.",
    tools=[TendenciasPrecoImoveisTool()],
    agent=analista_mercado,
    parameters=["cidade"],
    llm=llm
)

In [ ]:
analista_noticias= Agent(
    role="Analista de mercado",
    goal="Busca noticias relevantes sobre o mercado imobiliário para avaliar fatores externos",
    backstory="Especialista em analizar noticias e tendencias economicas que afetam os preços dos imóveis",
    verbose=True,
    max_iter=5,
    memory=True,
    llm=llm
)

In [ ]:
obter_noticias = Task(
    description="Busque notícias relevantes sobre o mercado imobiliário que possam impactar os preços dos imóveis.",
    expected_output="Lista de notícias relevantes sobre o mercado imobiliário.",
    agent=analista_noticias,
    parameters=["cidade"],
    llm=llm
)

In [ ]:
consultor_financeiro = Agent(
    role="Consultor Financeiro",
    goal="Fornecer recomendações financeiras personalizadas para a compra de imóveis, considerando o perfil do cliente.",
    backstory="Você é um consultor financeiro especializado em ajudar clientes a tomar decisões informadas sobre a compra de imóveis.",
    verbose=True,
    max_iter=5,
    allow_delegation=False,
    memory=True,
    llm=llm
)

In [ ]:
calcular_financiamento = Task(
    description="Analise a renda do cliente e sugira opções de financeamento viáveis.",
    expected_output="Tabela comparativa de opções de financiamento, taxa de juros e prazos.",
    agent=consultor_financeiro,
    llm=llm
)

In [ ]:
redator = Agent(
    role="Redator de Relatórios Imobiliários",
    goal="Criar relatórios detalhados sobre as tendências do mercado e imóveis encontrados.",
    backstory="""Você é um redator especializado em criar relatórios sobre o mercado imobiliário, incluindo análises de tendências e recomendações.
    Traduzindo dados complexos em informações acessíveis para os clientes.""",
    verbose=True,
    max_iter=5,
    allow_delegation=False,
    memory=True,
    llm=llm
)

In [ ]:
gerar_relatorio = Task(
    description="Crie um relatório detalhado sobre o melhor imóveis encontrado, considerando preços, tendências e financeamento.",
    expected_output="Relatório formatado com resumo do mercado, opções recomendadas e justificativa da escolha.",
    agent=redator,
    context=[buscar_imoveis, obter_tendencias, obter_noticias, calcular_financiamento],
    llm=llm
)

In [ ]:
Crew = Crew(
    agents=[corretor, analista_mercado, analista_noticias, consultor_financeiro, redator],
    tasks=[buscar_imoveis, obter_tendencias, obter_noticias, calcular_financiamento, gerar_relatorio],
    verbose=True,
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
    max_iter=15,
    manager_llm=llm,
    llm=llm
)

In [ ]:
result = Crew.kickoff(inputs={"cidade": "Rio de Janeiro", 
                             "imovel": "apartamento", 
                             "faixa_preco": "500000-700000"})

In [ ]:
from IPython.display import display, Markdown

In [ ]:
display(Markdown(str(result)))